In [ ]:
import boto3
import json
import time

region_aws = 'us-east-1'
ec2 = boto3.client('ec2', region_name=region_aws)

In [ ]:
# Create a key pair
key_pair_name = "my-keypair"
response = ec2.create_key_pair(KeyName=key_pair_name)

# Writing to file
with open(key_pair_name + ".pem", "w") as file1:
    file1.write(response['KeyMaterial'])
                         
print("Status code:", response['ResponseMetadata']['HTTPStatusCode'])

In [ ]:
!chmod 400 my-keypair.pem

En la terminal ejecute:

* `sam build`
* `sam deploy --guided --capabilities CAPABILITY_NAMED_IAM`

In [ ]:
print("Luego ejecute:\n")
print(">> sudo su")
print(">> cd /var/lib/mysql-files/")
print(">> ls -lrt")
print(">> mysql -u dbuser -pPassword@123")
print("  > use imdb;")
print("  > show tables;")

* Luego ejecute los comandos en el archivo `quer1.sql`

A título ilustrativo, a continuación se muestra un diagrama lógico que representa la relación entre varias tablas de origen que alojan el conjunto de datos IMDb.

* La tabla `title_basics` contiene las películas publicadas en EE.UU. después del año 2000. tconst es una clave alfanumérica asignada de forma exclusiva a cada película.
* `title_akas` contiene las regiones, los idiomas y los títulos de las películas publicadas. Es una relación 1:many con la tabla title_basics.
* `title_ratings` tiene la clasificación de las películas y el recuento de votos. Para este ejercicio, podemos suponer que la información se actualiza con alta frecuencia tras el estreno de la película. Tiene una relación 1:1 con la tabla title_basics
* `title_principals` contiene información sobre el reparto y el equipo. Tiene una relación 1:many con la tabla title_basics.
* `title_crew` tiene información sobre el guionista y el director. Tiene una relación 1:1 con la tabla title_basics.
* `name_basics` tiene información sobre el reparto y el equipo. Cada miembro tiene asignado un valor nconst único.
<br><br>

![Image](https://amazon-dynamodb-labs.com/images/migration31.jpg)

In [ ]:
!pip install mysql-connector-python -q

In [ ]:
import os
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np
from dynamodb import DynamoTable
import math
import json

In [ ]:
try:
    connection_config_dict = {
        'user': 'dbuser',
        'password': 'Password@123',
        'host': '35.174.107.243',
        'database': 'imdb',
        'raise_on_warnings': True,
        'use_pure': False,
        'autocommit': True,
        'pool_size': 32
    }
    connection = mysql.connector.connect(**connection_config_dict)

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Your connected to database: ", record)
        cursor = connection.cursor()
        # global connection timeout arguments
        global_connect_timeout = 'SET GLOBAL connect_timeout=655180'
        global_wait_timeout = 'SET GLOBAL connect_timeout=655180'
        global_interactive_timeout = 'SET GLOBAL connect_timeout=655180'

        cursor.execute(global_connect_timeout)
        cursor.execute(global_wait_timeout)
        cursor.execute(global_interactive_timeout)

        connection.commit()


except Error as e:
    print("Error while connecting to MySQL", e)

In [ ]:
cursor.execute("select count(*) from imdb.movies;")
total_movies = cursor.fetchone()
print("Total movies: ", total_movies[0])

In [ ]:
query = "select * from imdb.movies"
result_dataFrame = pd.read_sql(query, connection)

In [ ]:
cursor.close()
connection.close()
print("MySQL connection is closed")

In [ ]:
# result_dataFrame.to_csv('movies_compress.csv.gz', compression='gzip')

In [ ]:
df = pd.read_csv("movies_compress.csv.gz")
df = df.sort_values(by=['tconst']).reset_index().drop('Unnamed: 0', axis=1)
df = df.where(pd.notnull(df), None)

In [ ]:
df = df.iloc[0:30000]

In [ ]:
dt = DynamoTable(profile_name="my-profile", region="us-east-2")
try:
    dt.select_table('movies-d')
    print(dt)
except:
    dt.create_table(
        table_name='movies-d',
        partition_key='id',
        partition_key_type='S',
        sort_key="sk",
        sort_key_type="S"
)

In [ ]:
dm = df.iloc[0]
dm

In [ ]:
def generate_json(item):
    n_errors = []
    js = {
        "id": item["tconst"],
        "sk": item["nconst"],
    }
    if item["category"]: 
        js["category"] = item["category"]
    if item["genres"]: 
        js["genres"] = item["genres"].split(",")
    if item["job"]: 
        js["job"] = item["job"]    
    if item["characters"]:
        try:
            js["characters"] = json.loads(item["characters"])
        except:
            n_errors.append(item)
    if item["primaryName"]:
        js["primaryName"] = item["primaryName"]
    if item["writers"]: 
        js["writers"] = item["writers"].split(",")
    return js, n_errors

### Generate json files

In [ ]:
data = []
n_count = 0
n_end = 4000
n_pass = int(len(df) / n_end) + 1
n_breake = False

for p in range(1, n_pass+1):
    print(f"Pass {p} from {n_pass}")
    if (n_count + n_end) > len(df):
        n_c = n_count
        n_end = abs(n_c - len(df))
        print(n_count + n_end)
    for i in range(n_end):
        js, errors = generate_json(df.iloc[i+n_count])
        data.append(js)
    
    file_json = "json_data/data-" + str(p) + ".json"
    with open(file_json, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    data = []
    n_count += n_end

In [ ]:
files_json = os.listdir(path=os.getcwd() + "/json_data")
files_json.sort()
count_files_end = len(files_json)
for file in files_json:
    print(f"Process: {file}  -  Remaing: {count_files_end} files.")
    dt.load_json("json_data/" + file)
    count_files_end -= 1

In [ ]:
len(errors)